# SNR test

Find the maximum SNR where things still work

In [1]:
import sys
sys.path.insert(0, "../")

import matplotlib.pyplot as plt
import numpy as np
import uq4pk_src as uq4pk

In [2]:
from jax import random
rng_key = random.PRNGKey(32743)

In [3]:
np.random.seed(82011)

## Generate Mock Data

In [4]:
Theta_v_true = np.array([-150., 100., 1., 0., 0., -0.2, 0.05])
dv = 30.
lmd_min, lmd_max = 4000., 5000.
q = 15

ssps = uq4pk.model_grids.MilesSSP(lmd_min=lmd_min, lmd_max=lmd_max)
df = uq4pk.distribution_function.RandomGMM_DistributionFunction(modgrid=ssps, fill_pars={'n':2,
                                                                                         'logsig_lo':-2,
                                                                                         'logsig_hi':-1.5})
op = uq4pk.observation_operator.ObservationOperator(ssps=ssps, dv=dv)
ybar = op.evaluate(df.F, Theta_v_true)

Distribution functions should have shape (12, 53)


In [5]:
from uq4pk_fit.inference import StatModel
from simulate_data import load_experiment_data
data = load_experiment_data("experiment_data")
forward_operator = data.get_forward_operator()
fabian_model = StatModel(y=ybar, y_sd=np.ones_like(ybar), forward_operator=forward_operator)

/Users/pjethwa/miniconda3/envs/dcdirect/lib/python3.9/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


Distribution functions should have shape (12, 53)
Distribution functions should have shape (12, 53)


In [6]:
def get_input(snr):
    sigma_y = np.mean(np.abs(ybar))/snr
    y = np.random.normal(ybar, sigma_y)
    # fabian's rule of thumb
    regularisation_parameter = 1000.*snr
    beta_tilde_prior_cov = (1./regularisation_parameter) * np.dot(fabian_model.P1.mat, fabian_model.P1.mat.T)
    # adjust according to the expiriment in my notebook
    beta_tilde_prior_cov = 100.*beta_tilde_prior_cov
    return sigma_y, y, beta_tilde_prior_cov

# SNR=500

In [7]:
sigma_y, y, beta_tilde_prior_cov = get_input(500.)
svd_mcmc = uq4pk.svd_mcmc.SVD_MCMC(
    ssps=ssps,
    Theta_v_true=Theta_v_true,
    y=y,
    ybar=ybar,
    sigma_y=sigma_y)
svd_mcmc.set_q(q)

Distribution functions should have shape (12, 53)


Sample eta and alpha

In [8]:
eta_alpha_model = svd_mcmc.get_eta_alpha_model(sigma_alpha=0.1, sigma_eta=0.1)
eta_alpha_sampler = svd_mcmc.get_mcmc_sampler(eta_alpha_model, num_warmup=500, num_samples=500)
eta_alpha_sampler.run(rng_key)
eta_alpha_sampler.print_summary()

sample: 100%|█████████████████| 1000/1000 [00:06<00:00, 158.90it/s, 127 steps of size 3.04e-03. acc. prob=0.93]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
     alpha      1.00      0.00      1.00      1.00      1.00    448.80      1.00
    eta[0]     -0.01      0.00     -0.01     -0.01     -0.01    522.86      1.00
    eta[1]     -0.00      0.00     -0.00     -0.00      0.00    678.92      1.00
    eta[2]     -0.03      0.00     -0.03     -0.03     -0.03    611.87      1.00
    eta[3]      0.01      0.00      0.01      0.00      0.01    576.53      1.00
    eta[4]      0.01      0.00      0.01      0.01      0.01    411.41      1.00
    eta[5]      0.01      0.00      0.01      0.01      0.01    343.59      1.00
    eta[6]     -0.00      0.00     -0.00     -0.00     -0.00    238.72      1.00
    eta[7]      0.01      0.00      0.01      0.01      0.02    244.80      1.00
    eta[8]      0.03      0.00      0.03      0.03      0.04    210.58      1.00
    eta[9]      0.03      0.00      0.03      0.03      0.04    571.09      1.00
   eta[10]      0.03      0

Sample beta

In [9]:
eta_alpha_samples = eta_alpha_sampler.get_samples()
beta_tilde_model = svd_mcmc.get_beta_tilde_model(
    eta_alpha_samples=eta_alpha_samples,
    Sigma_beta_tilde=beta_tilde_prior_cov)
beta_tilde_sampler = svd_mcmc.get_mcmc_sampler(beta_tilde_model, num_warmup=50, num_samples=50)
beta_tilde_sampler.run(rng_key)
beta_tilde_sampler.print_summary()

sample: 100%|█████████████████████| 100/100 [00:06<00:00, 15.56it/s, 47 steps of size 6.16e-04. acc. prob=0.16]



                     mean       std    median      5.0%     95.0%     n_eff     r_hat
  beta_tilde[0]      0.00      0.00      0.00      0.00      0.00     10.41      1.12
  beta_tilde[1]      0.00      0.00      0.00      0.00      0.00     10.32      1.15
  beta_tilde[2]      0.00      0.00      0.00      0.00      0.00      8.71      1.16
  beta_tilde[3]      0.00      0.00      0.00      0.00      0.00      9.57      1.11
  beta_tilde[4]      0.00      0.00      0.00      0.00      0.00      9.80      1.10
  beta_tilde[5]      0.00      0.00      0.00      0.00      0.00      4.90      1.44
  beta_tilde[6]      0.00      0.00      0.00      0.00      0.00      6.42      1.38
  beta_tilde[7]      0.00      0.00      0.00      0.00      0.00     12.84      1.08
  beta_tilde[8]      0.00      0.00      0.00      0.00      0.00      8.55      1.15
  beta_tilde[9]      0.00      0.00      0.00      0.00      0.00     16.34      1.13
 beta_tilde[10]      0.00      0.00      0.00      0.

# SNR=1000

In [10]:
sigma_y, y, beta_tilde_prior_cov = get_input(100.)
svd_mcmc = uq4pk.svd_mcmc.SVD_MCMC(
    ssps=ssps,
    Theta_v_true=Theta_v_true,
    y=y,
    ybar=ybar,
    sigma_y=sigma_y)
svd_mcmc.set_q(q)

Distribution functions should have shape (12, 53)


Sample eta and alpha

In [11]:
eta_alpha_model = svd_mcmc.get_eta_alpha_model(sigma_alpha=0.1, sigma_eta=0.1)
eta_alpha_sampler = svd_mcmc.get_mcmc_sampler(eta_alpha_model, num_warmup=500, num_samples=500)
eta_alpha_sampler.run(rng_key)
eta_alpha_sampler.print_summary()

sample: 100%|█████████████████| 1000/1000 [00:05<00:00, 167.46it/s, 127 steps of size 1.43e-02. acc. prob=0.94]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
     alpha      1.00      0.00      1.00      1.00      1.00    328.08      1.01
    eta[0]     -0.01      0.00     -0.01     -0.01     -0.01    471.00      1.00
    eta[1]      0.00      0.00      0.00     -0.00      0.00    581.44      1.00
    eta[2]     -0.03      0.00     -0.03     -0.03     -0.03    731.29      1.00
    eta[3]      0.00      0.00      0.00      0.00      0.01    522.46      1.01
    eta[4]      0.01      0.00      0.01      0.01      0.02    248.40      1.00
    eta[5]      0.01      0.00      0.01      0.00      0.01    326.47      1.00
    eta[6]     -0.01      0.01     -0.01     -0.02      0.00    135.45      1.02
    eta[7]      0.01      0.01      0.01     -0.00      0.02    510.20      1.00
    eta[8]      0.02      0.01      0.02      0.00      0.04    825.09      1.00
    eta[9]      0.02      0.01      0.02     -0.00      0.03    703.43      1.00
   eta[10]      0.03      0

Sample beta

In [12]:
eta_alpha_samples = eta_alpha_sampler.get_samples()
beta_tilde_model = svd_mcmc.get_beta_tilde_model(
    eta_alpha_samples=eta_alpha_samples,
    Sigma_beta_tilde=beta_tilde_prior_cov)
beta_tilde_sampler = svd_mcmc.get_mcmc_sampler(beta_tilde_model, num_warmup=50, num_samples=50)
beta_tilde_sampler.run(rng_key)
beta_tilde_sampler.print_summary()

sample: 100%|███████████████████| 100/100 [00:27<00:00,  3.64it/s, 1023 steps of size 1.24e-03. acc. prob=0.96]


                     mean       std    median      5.0%     95.0%     n_eff     r_hat
  beta_tilde[0]      0.00      0.00      0.00      0.00      0.00     32.40      1.01
  beta_tilde[1]      0.00      0.00      0.00      0.00      0.00     46.23      0.98
  beta_tilde[2]      0.00      0.00      0.00      0.00      0.00     47.25      0.98
  beta_tilde[3]      0.00      0.00      0.00      0.00      0.00     34.42      1.02
  beta_tilde[4]      0.00      0.00      0.00      0.00      0.00     40.59      0.99
  beta_tilde[5]      0.00      0.00      0.00      0.00      0.00     18.92      0.99
  beta_tilde[6]      0.00      0.00      0.00      0.00      0.00     37.21      1.01
  beta_tilde[7]      0.00      0.00      0.00      0.00      0.00     26.91      0.99
  beta_tilde[8]      0.00      0.00      0.00      0.00      0.00     42.93      0.99
  beta_tilde[9]      0.00      0.00      0.00      0.00      0.00     14.98      1.17
 beta_tilde[10]      0.00      0.00      0.00      0.

# SNR=1500

In [13]:
sigma_y, y, beta_tilde_prior_cov = get_input(1500.)
svd_mcmc = uq4pk.svd_mcmc.SVD_MCMC(
    ssps=ssps,
    Theta_v_true=Theta_v_true,
    y=y,
    ybar=ybar,
    sigma_y=sigma_y)
svd_mcmc.set_q(q)

Distribution functions should have shape (12, 53)


Sample eta and alpha

In [14]:
eta_alpha_model = svd_mcmc.get_eta_alpha_model(sigma_alpha=0.1, sigma_eta=0.1)
eta_alpha_sampler = svd_mcmc.get_mcmc_sampler(eta_alpha_model, num_warmup=500, num_samples=500)
eta_alpha_sampler.run(rng_key)
eta_alpha_sampler.print_summary()

sample: 100%|█████████████████| 1000/1000 [00:29<00:00, 33.60it/s, 1023 steps of size 6.97e-09. acc. prob=0.71]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
     alpha      1.00      0.00      1.00      1.00      1.00      0.50      1.00
    eta[0]     -0.01      0.00     -0.01     -0.01     -0.01      0.50      1.00
    eta[1]      0.00      0.00      0.00      0.00      0.00      3.78      1.37
    eta[2]     -0.02      0.00     -0.02     -0.02     -0.02      0.50      1.00
    eta[3]     -0.27      0.00     -0.27     -0.27     -0.27      0.50      1.00
    eta[4]      0.02      0.00      0.02      0.02      0.02      0.50      1.00
    eta[5]      1.44      0.00      1.44      1.44      1.44      0.50      1.00
    eta[6]     -0.24      0.00     -0.24     -0.24     -0.24      0.50      1.00
    eta[7]      1.04      0.00      1.04      1.04      1.04      0.50      1.00
    eta[8]     -1.69      0.00     -1.69     -1.69     -1.69      0.50      1.00
    eta[9]     -0.37      0.00     -0.37     -0.37     -0.37      0.50      1.00
   eta[10]      0.58      0

Sample beta

In [15]:
eta_alpha_samples = eta_alpha_sampler.get_samples()
beta_tilde_model = svd_mcmc.get_beta_tilde_model(
    eta_alpha_samples=eta_alpha_samples,
    Sigma_beta_tilde=beta_tilde_prior_cov)
beta_tilde_sampler = svd_mcmc.get_mcmc_sampler(beta_tilde_model, num_warmup=50, num_samples=50)
beta_tilde_sampler.run(rng_key)
beta_tilde_sampler.print_summary()

ValueError: Normal distribution got invalid scale parameter.